In [8]:
"""
This script performs the following tasks:
1. Imports necessary libraries and modules:
    - `os`: Provides a way of using operating system dependent functionality.
    - `pathlib.Path`: Offers classes representing filesystem paths with semantics appropriate for different operating systems.
    - `requests`: Allows sending HTTP requests.
    - `bs4.BeautifulSoup`: Facilitates web scraping by parsing HTML and XML documents.
    - `icalendar.Calendar`, `icalendar.Event`: Enables creation and manipulation of iCalendar files.
    - `datetime.datetime`, `datetime.timedelta`: Supplies classes for manipulating dates and times.
    - `pytz`: Brings the Olson tz database into Python and allows accurate and cross platform timezone calculations.
    - `xml.etree.ElementTree`: Implements a simple and efficient API for parsing and creating XML data.
"""
import os
from pathlib import Path
import requests
from bs4 import BeautifulSoup
from icalendar import Calendar, Event
from datetime import datetime, timedelta
import pytz
import xml.etree.ElementTree as ET


In [9]:
def scrape_onefootball_fixtures():
    """
    Scrapes Premier League fixtures from the OneFootball website.

    This function sends a GET request to the OneFootball Premier League fixtures page,
    parses the HTML content using BeautifulSoup, and extracts fixture details such as
    date, time, home team, away team, and match status.

    Returns:
        list: A list of dictionaries, each containing details of a fixture.
              Each dictionary has the following keys:
              - 'date': The date of the match in "dd/mm/yyyy" format.
              - 'time': The time of the match in "HH:MM" format.
              - 'home_team': The name of the home team.
              - 'away_team': The name of the away team.
              - 'status': The status of the match (e.g., "Scheduled", "Postponed").

    Example:
        fixtures = scrape_onefootball_fixtures()
        for fixture in fixtures:
            print(f"Date: {fixture['date']}, Time: {fixture['time']}, Home Team: {fixture['home_team']}, Away Team: {fixture['away_team']}, Status: {fixture['status']}")
    """
    url = "https://onefootball.com/en/competition/premier-league-9/fixtures"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    fixtures = []
    
    for match_day in soup.find_all('ul', class_='MatchCardsList_matches__8_UwB'):
        # Extract matches within the match day
        for match in match_day.find_all('a', class_='MatchCard_matchCard__iOv4G'):
            # Extract home and away teams
            home_team = match.find('span', class_='SimpleMatchCardTeam_simpleMatchCardTeam__name__7Ud8D').text.strip()
            away_team = match.find_all('span', class_='SimpleMatchCardTeam_simpleMatchCardTeam__name__7Ud8D')[1].text.strip()
            
            # Extract date and time from the <time> elements
            time_element = match.find('time', class_='title-8-bold')
            if time_element:
                match_time = time_element['datetime']  # Get the datetime attribute
                match_date = datetime.fromisoformat(match_time[:-1])  # Remove 'Z' and convert to datetime
                formatted_date = match_date.strftime("%d/%m/%Y")
                formatted_time = match_date.strftime("%H:%M")
            else:
                formatted_date = 'Unknown Date'
                formatted_time = 'Unknown Time'

            # Extract status if available
            status_element = match.find('span', class_='SimpleMatchCard_simpleMatchCard__infoMessage___NJqW')
            status = status_element.text.strip() if status_element else 'Unknown Status'
            
            print(f"Date: {formatted_date}, Time: {formatted_time}, Home Team: {home_team}, Away Team: {away_team}, Status: {status}")
            
            fixtures.append({
                'date': formatted_date,
                'time': formatted_time,
                'home_team': home_team,
                'away_team': away_team,
                'status': status
            })
    
    return fixtures

In [11]:
def create_ics_file(fixtures, filename):
    """
    Creates an ICS file from the given fixtures.

    This function initializes a calendar, iterates through the provided fixtures,
    and creates calendar events for each fixture. Each event includes details such as
    the home team, away team, match date, time, and status. The events are then added
    to the calendar, which is saved as an ICS file.

    Args:
        fixtures (list): A list of dictionaries, each containing details of a fixture.
                         Each dictionary should have the following keys:
                         - 'date': The date of the match in "dd/mm/yyyy" format.
                         - 'time': The time of the match in "HH:MM" format.
                         - 'home_team': The name of the home team.
                         - 'away_team': The name of the away team.
                         - 'status': The status of the match (e.g., "Scheduled", "Postponed").
        filename (str): The name of the ICS file to be created.

    Example:
        fixtures = [
            {
                'date': '01/01/2023',
                'time': '15:00',
                'home_team': 'Team A',
                'away_team': 'Team B',
                'status': 'Scheduled'
            },
            # Add more fixtures as needed
        ]
        create_ics_file(fixtures, 'premier_league_fixtures.ics')
    """
    # Initialize the calendar
    cal = Calendar()

    for fixture in fixtures:
        event = Event()
        event.add('summary', f"{fixture['home_team']} vs {fixture['away_team']}")

        # Combine date and time into a single datetime object
        date_str = f"{fixture['date']} {fixture['time']}"
        start_time = datetime.strptime(date_str, "%d/%m/%Y %H:%M")
        start_time = pytz.utc.localize(start_time)

        # Add event details
        event.add('dtstart', start_time)
        event.add('dtend', start_time + timedelta(hours=2))  # Assume 2 hours duration
        event.add('description', f"Status: {fixture['status']}")

        # Add the event to the calendar
        cal.add_component(event)

    # Create directory if it doesn't exist
    directory = Path.cwd() / 'ICS_Files'
    directory.mkdir(parents=True, exist_ok=True)

    # Write to disk
    with open(os.path.join(directory, filename), 'wb') as f:
        f.write(cal.to_ical())

In [12]:
def main():
    """
    Main function to scrape Premier League fixtures and create an ICS file.

    This function calls the `scrape_onefootball_fixtures` function to scrape
    the fixtures from the OneFootball website, prints the scraped fixtures,
    and then calls the `create_ics_file` function to create an ICS file with
    the scraped fixtures.

    Example:
        main()
    """
    fixtures = scrape_onefootball_fixtures()
    print(f"fixtures: {fixtures}")
    create_ics_file(fixtures, 'premier_league_fixtures.ics')
    print(f"Created ICS file with {len(fixtures)} fixtures.")

In [13]:
if __name__ == "__main__":
    """
    Entry point for the script.

    This block ensures that the `main` function is called only when the script
    is run directly, and not when it is imported as a module in another script.

    Example:
        To run the script, execute the following command in the terminal:
        python script_name.py
    """
    main()

Date: 28/09/2024, Time: 11:30, Home Team: Newcastle United, Away Team: Manchester City, Status: Full time
Date: 28/09/2024, Time: 14:00, Home Team: Arsenal, Away Team: Leicester City, Status: Full time
Date: 28/09/2024, Time: 14:00, Home Team: Nottingham Forest, Away Team: Fulham, Status: Full time
Date: 28/09/2024, Time: 14:00, Home Team: Everton, Away Team: Crystal Palace, Status: Full time
Date: 28/09/2024, Time: 14:00, Home Team: Brentford, Away Team: West Ham United, Status: Full time
Date: 28/09/2024, Time: 14:00, Home Team: Chelsea, Away Team: Brighton & Hove Albion, Status: Full time
Date: 28/09/2024, Time: 16:30, Home Team: Wolverhampton Wanderers, Away Team: Liverpool FC, Status: Full time
Date: 29/09/2024, Time: 13:00, Home Team: Ipswich Town, Away Team: Aston Villa, Status: Full time
Date: 29/09/2024, Time: 15:30, Home Team: Manchester United, Away Team: Tottenham Hotspur, Status: Full time
Date: 30/09/2024, Time: 19:00, Home Team: AFC Bournemouth, Away Team: Southampton, S